In [6]:
import pandas as pd
import requests
import uuid
import re
import datetime
from datetime import date
from bs4 import BeautifulSoup


def get_date_speech(year_session, link_speech, text_day):
    day_speech = re.findall('[0-9]+', text_day)[0]
    month_speech = month_to_number(link_speech.split('/')[-2:][0])
    return(datetime.date(int(year_session), int(month_speech), int(day_speech)))
def month_to_number(m):
    if m == 'ene':
        return(1)
    elif m == 'feb':
        return(2)
    elif m == 'mar':
        return(3)
    elif m == 'abr':
        return(4)
    elif m == 'may':
        return(5)
    elif m == 'jun':
        return(6)
    elif m == 'jul':
        return(7)
    elif m == 'ago':
        return(8)
    elif m == 'sep':
        return(9)
    elif m == 'oct':
        return(10)
    elif m == 'nov':
        return(11)
    elif m == 'dic':
        return(12)
def get_links_legislaturas(legislaturas):
    """
    Parametros:
    -----------
    legislatura: list of int numbers of legislatura

    Returns:
    --------
    lista de diccionario con los a el htmls de los debates legislativos
    """
    all_data = []

    for legislatura in legislaturas:
        link_debates = 'http://cronica.diputados.gob.mx/DDebates/' + str(legislatura) + '/index.html'

        r = requests.get(link_debates)
        soup = BeautifulSoup(r.content, 'html.parser')

        links_sesiones = []
        type_sesion = []
        urls = []
        for h in soup.findAll('a'):
            links_sesiones.append('http://cronica.diputados.gob.mx/DDebates/' + str(legislatura) + '/' + h.get('href'))
            type_sesion.append(re.sub('\xa0', '', h.text))
            urls.append(h.get('href'))


        for x, y, urls in zip(links_sesiones, type_sesion, urls):
            r = requests.get(x)
            soup = BeautifulSoup(r.content, 'html.parser')

            for i in soup.findAll('a'):
                year_session = re.findall('[0-9]{4}', soup.findAll('tbody')[1].find('tr').text)[0]
                link_ = 'http://cronica.diputados.gob.mx/DDebates/' + str(legislatura)+'/'+ re.sub('index[.]html', '', urls) +i.get('href')

                fecha_speech = get_date_speech(year_session, link_, i.text)

                temp = {}
                temp['id_debate'] = str(uuid.uuid3(uuid.NAMESPACE_URL,link_))
                temp['legislatura'] = legislatura
                temp['date_debate'] = fecha_speech
                temp['type_sesion'] = y
                temp['link_debate'] = link_
                all_data.append(temp)
    return(all_data)
def obtain_texto_debate(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    contenido_debate = soup.findAll("div", {"class": "Contenido"})[0].text

    dict_contenidos = {}
    for i in contenido_debate.split('\n\n\n \n'):
        try:
            name_tema = re.sub('\n', '', re.findall('^[A-Z].*\n',i)[0])
            text_all = re.sub(name_tema, '', i)
            text_all = re.sub('[ ]+$', '', re.sub('^[ ]+', '', re.sub('[ ]+', ' ', re.sub('\n+', ' ', text_all))))
            dict_contenidos[name_tema] = text_all
        except:
            continue
    dict_contenidos['id_debate'] = str(uuid.uuid3(uuid.NAMESPACE_URL,link))
    #main_text = []
    #for x in dict_contenidos.values():
    #    main_text.append(x)
    #return(re.sub('\n', ' ', ' '.join(main_text)))
    #print(len(dict_contenidos.keys()))
    return(dict_contenidos)
def obtain_text_sesion(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')

    try:
        links_volumenes = []
        for h in soup.findAll('a'):
            if any(re.findall('Volúmen', h.text)):
                links_volumenes.append(h.get('href'))

        real_link = '/'.join(link.split('/')[:-1]) + '/' + links_volumenes[0]

        texto_debate = obtain_texto_debate(real_link)
        return(texto_debate)
    except:
        return(obtain_texto_debate(link))


In [7]:
def obtain_speaker_and_text(x):
    speaker = re.sub('[0-9]+[:][0-9]+', 'HORA', x).split(':')[0]
    text = re.sub('[0-9]+[:][0-9]+', 'HORA', x).split(':')[1]
    speaker = re.sub('[(].*', '', speaker)
    speaker = re.sub('^[ ]+', '', speaker)
    return(speaker, text)

In [9]:
def df_speaker_speech(dict_data):
    strings = ['La Presidenta diputada', 'La Secretaria diputada', 
                'El secretario general', 'El Presidente diputado', 'El Secretario diputado', 
                '[.] El diputado', '[.] La diputada']
    
    
    all_text = []
    for i in dict_data.values():
        all_text.append(i)
    all_text = ' '.join(all_text)
    all_interventions = []
    for i in re.split(r'|'.join(strings), all_text):
        if not any(i):
            #print(i)
            continue
        else:
            try:
                speaker, test = obtain_speaker_and_text2(i)
                temp = {}
                temp['speaker'] = speaker
                temp['speech'] = test
                all_interventions.append(temp)
            except:
                print('error')
    df = pd.DataFrame(all_interventions)
    return(df)

In [11]:
link_debates = 'http://cronica.diputados.gob.mx/DDebates/' + str(60) + '/index.html'

In [14]:
get_links_legislaturas(60)

TypeError: 'int' object is not iterable

In [15]:
data = get_links_legislaturas([60])

IndexError: list index out of range

In [6]:
df_data = pd.DataFrame(data)

In [55]:
link_debate = df_data.link_debate[0]

In [56]:
link_debate

'http://cronica.diputados.gob.mx/DDebates/63/1er/1P/Ord/ago/01L63A1P101.html'

In [57]:
r = requests.get(link_debate)
soup = BeautifulSoup(r.content, 'html.parser')
contenido_debate = soup.findAll("div", {"class": "Contenido"})[0].text

In [61]:
re.findall('Se abre la sesion', contenido_debate)

[]

In [62]:
contenido_debate

'\n\n\n \nINFORME DE LA SECRETARÍA GENERAL\n\nEl secretario general Mauricio Farah Gebara(a las 11:31 horas):\nHonorable asamblea, señoras diputadas federales electas, señores diputados\nfederales electos, de conformidad con lo dispuesto en el artículo 14 de la Ley\nOrgánica del Congreso General de los Estados Unidos Mexicanos y a efecto de la\nrealización de esta sesión, entre los días 20 y 28 de agosto se registraron\nante la Secretaría General de esta soberanía, 496 diputadas y diputados\nelectos, es decir, la mayoría de sus integrantes.\nDe igual forma, la asistencia de los presentes el día de hoy atiende a\nlo dispuesto en el artículo 63 de la Constitución Política de los Estados\nUnidos Mexicanos.\nEn consecuencia, puede dar inicio la sesión constitutiva de la Cámara\nde Diputados a la LXIII Legislatura del Honorable Congreso de la Unión de los\nEstados Unidos Mexicanos conforme al siguiente orden del día.\nUno. Informe de la Secretaría General e integración de la Mesa de\nDecano

In [48]:
elements = []
for i in contenido_debate.split('\n\n\n \n'):
    if any(i):
        for j in i.split('\n\n'):
            if not any(re.findall('[a-z]+', j)):
                print(j)
            else:
                elements.append(j)
    else:
        continue

INFORME DE LA SECRETARÍA GENERAL
INTEGRACIÓN DE LA MESA DE DECANOS

DECLARACIÓN DE QUORUM
ORDEN DEL DÍA
PROTESTA CONSTITUCIONAL DE LA PRESIDENTA DE LA MESA DE DECANOS

PROTESTA CONSTITUCIONAL DE LAS DIPUTADAS Y LOS DIPUTADOS ELECTOS A LA SEXAGÉSIMA TERCERA LEGISLATURA
ELECCIÓN DE LOS INTEGRANTES DE LA MESA DIRECTIVA QUE FUNCIONARÁ DURANTE EL PRIMER AÑO DE EJERCICIO DE LA SEXAGÉSIMA TERCERA LEGISLATURA
TOMA DE PROTESTA DE LA MESA DIRECTIVA

DECLARACIÓN DE LA LEGAL CONSTITUCIÓN DE LA SEXAGÉSIMA TERCERA LEGISLATURA DE LA CÁMARA DE DIPUTADOS
MENSAJE DE LA PRESIDENCIA DE LA MESA DIRECTIVA
HIMNO NACIONAL
CITA



In [43]:
speakers = []
texts = []
for i in elements:
    x = re.sub('[0-9]+[:][0-9]+', 'HORA', i)
    speaker = x.split(':')[0]
    text = ' '.join(x.split(':')[1:])
    
    speakers.append(speaker)
    texts.append(text)

In [47]:
pd.DataFrame({'speaker':speaker,
    'text':texts})

,speaker,text
0,El Presidente diputado José de Jesús Zambrano ...,"\nHonorable asamblea, señoras diputadas federa..."
1,El Presidente diputado José de Jesús Zambrano ...,"Esta Presidencia, sus vicepresidentes, las se..."
2,El Presidente diputado José de Jesús Zambrano ...,"Con su venia, señora presidenta. Se informa a ..."
3,El Presidente diputado José de Jesús Zambrano ...,Hoy 29 de agosto de 2015 se abre la Sesión\nC...
4,El Presidente diputado José de Jesús Zambrano ...,«Sesión\nConstitutiva de la Cámara de Diputado...
5,El Presidente diputado José de Jesús Zambrano ...,
6,El Presidente diputado José de Jesús Zambrano ...,
7,El Presidente diputado José de Jesús Zambrano ...,Protesto guardar y hacer guardar la Constituc...
8,El Presidente diputado José de Jesús Zambrano ...,"\nDiputadas y diputados electos, ¿Protestan gu..."
9,El Presidente diputado José de Jesús Zambrano ...,Sí\nprotesto.


In [10]:
contenido_debate.split('\n\n')[23]

'La Secretaria diputada María Marcela González Salas y Petricioli: Acuerdo de los Coordinadores de los Grupos Parlamentarios,\nrelativo a la integración y elección de la Mesa Directiva de la Cámara de\nDiputados para el primer año de ejercicio de la LXIII Legislatura.\n[Acuerdo]'

In [101]:
re.split('[A-Z]+\n\n', contenido_debate.split('\n\n\n \n')[1])

['INFORME DE LA SECRETARÍA ',
 'El secretario general Mauricio Farah Gebara(a las 11:31 horas):\nHonorable asamblea, señoras diputadas federales electas, señores diputados\nfederales electos, de conformidad con lo dispuesto en el artículo 14 de la Ley\nOrgánica del Congreso General de los Estados Unidos Mexicanos y a efecto de la\nrealización de esta sesión, entre los días 20 y 28 de agosto se registraron\nante la Secretaría General de esta soberanía, 496 diputadas y diputados\nelectos, es decir, la mayoría de sus integrantes.\nDe igual forma, la asistencia de los presentes el día de hoy atiende a\nlo dispuesto en el artículo 63 de la Constitución Política de los Estados\nUnidos Mexicanos.\nEn consecuencia, puede dar inicio la sesión constitutiva de la Cámara\nde Diputados a la LXIII Legislatura del Honorable Congreso de la Unión de los\nEstados Unidos Mexicanos conforme al siguiente orden del día.\nUno. Informe de la Secretaría General e integración de la Mesa de\nDecanos.\nDos. Decla

In [115]:
elements = []
for i in contenido_debate.split('\n\n\n \n'):
    if not any(i):
        continue
    else:
        for j in i.split('\n\n'):
            if any(j):
                elements.append(j)
            else:
                continue
        #elements.append(i.split('\n\n'))

In [116]:
len(elements)

41

In [174]:
interventions[0]

['El secretario general Mauricio Farah Gebara(a las  horas)',
 '\nHonorable asamblea, señoras diputadas federales electas, señores diputados\nfederales electos, de conformidad con lo dispuesto en el artículo 14 de la Ley\nOrgánica del Congreso General de los Estados Unidos Mexicanos y a efecto de la\nrealización de esta sesión, entre los días 20 y 28 de agosto se registraron\nante la Secretaría General de esta soberanía, 496 diputadas y diputados\nelectos, es decir, la mayoría de sus integrantes.\nDe igual forma, la asistencia de los presentes el día de hoy atiende a\nlo dispuesto en el artículo 63 de la Constitución Política de los Estados\nUnidos Mexicanos.\nEn consecuencia, puede dar inicio la sesión constitutiva de la Cámara\nde Diputados a la LXIII Legislatura del Honorable Congreso de la Unión de los\nEstados Unidos Mexicanos conforme al siguiente orden del día.\nUno. Informe de la Secretaría General e integración de la Mesa de\nDecanos.\nDos. Declaración de quórum.\nTres. Protes

In [172]:
interventions = []
for i in elements:
    x = re.sub('[0-9]+[:][0-9]+', '', i)
    if len(x.split(':')) == 1:
        print(i)
    else:
        interventions.append([x.split(':')[0], ' '.join(x.split(':')[1:])])

INFORME DE LA SECRETARÍA GENERAL
INTEGRACIÓN DE LA MESA DE DECANOS
DECLARACIÓN DE QUORUM
ORDEN DEL DÍA
Orden del día
Sábado 29 de agosto de 2015.
Informe de la Secretaría General e integración de la Mesa de
Decanos.
Declaración de quórum.
Protesta constitucional del presidente de la Mesa de Decanos.
Protesta constitucional de los diputados electos a la LXIII
Legislatura.
Elección de los integrantes de la Mesa Directiva que funcionará durante
el primer año de ejercicio de la LXIII Legislatura.
Declaración de la legal constitución de la Cámara de Diputados de la
LXIII Legislatura.
Cita a sesión de Congreso General.
Designación de comisiones reglamentarias.»
Se invita a los presentes a ponerse de pie.
PROTESTA CONSTITUCIONAL DE LA PRESIDENTA DE LA MESA DE DECANOS
PROTESTA CONSTITUCIONAL DE LAS DIPUTADAS Y LOS DIPUTADOS ELECTOS A LA SEXAGÉSIMA TERCERA LEGISLATURA
Les rogamos tomar sus asientos de nuevo, señoras y señores diputados.
Continuaremos con el orden del día. Les rogamos pasen a to

In [170]:
interventions[1]

['El secretario general Mauricio Farah Gebara(a las  horas)',
 '\nHonorable asamblea, señoras diputadas federales electas, señores diputados\nfederales electos, de conformidad con lo dispuesto en el artículo 14 de la Ley\nOrgánica del Congreso General de los Estados Unidos Mexicanos y a efecto de la\nrealización de esta sesión, entre los días 20 y 28 de agosto se registraron\nante la Secretaría General de esta soberanía, 496 diputadas y diputados\nelectos, es decir, la mayoría de sus integrantes.\nDe igual forma, la asistencia de los presentes el día de hoy atiende a\nlo dispuesto en el artículo 63 de la Constitución Política de los Estados\nUnidos Mexicanos.\nEn consecuencia, puede dar inicio la sesión constitutiva de la Cámara\nde Diputados a la LXIII Legislatura del Honorable Congreso de la Unión de los\nEstados Unidos Mexicanos conforme al siguiente orden del día.\nUno. Informe de la Secretaría General e integración de la Mesa de\nDecanos.\nDos. Declaración de quórum.\nTres. Protes

In [128]:
re.findall('[A-Z]+',elements[14] )

['L',
 'P',
 'M',
 'E',
 'J',
 'S',
 'L',
 'D',
 'P',
 'C',
 'P',
 'E',
 'U',
 'M',
 'LXIII',
 'L',
 'C',
 'U']

In [7]:
from tqdm import tqdm

In [8]:
list_dict_sessions = []
errors = []

In [9]:
for i in tqdm(list(df_data.link_debate)):
    try:
        list_dict_sessions.append(obtain_text_sesion(i))
    except:
        errors.append(i)

100%|██████████| 223/223 [05:01<00:00,  1.35s/it]


In [11]:
len(list_dict_sessions)

72

In [18]:
all_text = []
for i in list_dict_sessions[0].values():
    all_text.append(i)
all_text = ' '.join(all_text)

In [17]:
df_speaker_speech(list_dict_sessions[0])

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error


""


In [97]:
df_speeches = []
for i in tqdm(range(0, len(list_dict_sessions))):
    df_speeches.append(df_speaker_speech(list_dict_sessions[i]))

  3%|▎         | 7/221 [00:00<00:04, 52.26it/s]

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
erro

  4%|▍         | 9/221 [00:00<00:09, 22.38it/s]

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
erro

  6%|▌         | 13/221 [00:00<00:10, 20.30it/s]

...

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
erro

 93%|█████████▎| 206/221 [00:07<00:00, 51.61it/s]

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
erro

100%|██████████| 221/221 [00:08<00:00, 27.07it/s]


error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
err

In [47]:
list(pd.concat(df_speeches).speaker.unique())

['Mauricio Farah Gebara',
 'electa María Esther de Jesús Scherman Leaño',
 'electa María Marcela González Salas y Petricioli',
 'Noemí Zoila Guzmán Lagunes',
 'María Esther de Jesús Scherman Leaño',
 'María Marcela González Salas y Petricioli',
 'Alejandra Noemí Reynoso Sánchez',
 'José de Jesús Zambrano Grijalva',
 'Ramón Bañales Arámbula',
 'José de Jesús Zambrano Grijalva \xa0',
 'José de Jesús Zambrano Grijalva\xa0',
 'Manuel Jesús Clouthier Carrillo',
 'Hugo Eric Flores Cervantes',
 'Luis Alfredo Valles Mendoza',
 'José Clemente Castañeda Hoeflich',
 'Norma Rocío Nahle García',
 'Francisco Martínez Neri',
 'Marko Antonio Cortés Mendoza',
 'Jorge Carlos Ramírez Marín',
 'Jorge Carlos Ramírez Marín ',
 'Yulma Rocha Aguilar',
 'Ana Guadalupe Perea Santos',
 'Ernestina Godoy Ramos',
 'Juan Manuel Celis Aguirre',
 'Claudia Sofía Corichi García',
 'María Elena Orantes López ',
 'Esthela de Jesús Ponce Beltrán',
 'Minerva Hernández Ramos',
 'Marko Antonio Cortés Mendoza ',
 'Araceli Dami

In [34]:
all_text = []
for i in list_dict_sessions[32].values():
    all_text.append(i)
all_text = ' '.join(all_text)

In [35]:
strings = ['La Presidenta diputada', 'La Secretaria diputada', 
                'El secretario general', 'El Presidente diputado', 'El Secretario diputado', 
                '[.] El diputado', '[.] La diputada']
all_interventions = []
for i in tqdm(re.split(r'|'.join(strings), all_text)):
    if not any(i):
        continue
    else:
        try:
            speaker, test = obtain_speaker_and_text2(i)
            temp = {}
            temp['speaker'] = speaker
            temp['speech'] = test
            all_interventions.append(temp)
        except:
            break

 44%|████▍     | 160/364 [00:00<00:00, 9017.58it/s]


In [38]:
re.findall('El diputado .* [(].*[)][:]', all_text)[0]

'El diputado Guillermo Rafael Santiago Rodríguez (desde la curul): Presidente. El Presidente diputado José de Jesús Zambrano Grijalva: Sonido en la curul del diputado Guillermo Santiago, por favor. El diputado Guillermo Rafael Santiago Rodríguez (desde la curul): Sí, presidente, para mencionar y expresar que es increíble el silencio que ha guardado este pleno ante lo que ha ocurrido en Chiapas y en otros estados de la República. El día de ayer falleció un maestro por la situación tan hostil que está viviendo nuestro estado, cómo es posible que por cada maestro evaluado llevaron a más de 10 policías federales. Nuevamente hago el exhorto desde mi curul, para que el gobierno el gobernador Manuel Velasco Coello y el gobierno federal cesen ya con la represión y con este hostigamiento y violación de los derechos humanos de las y los maestros. Gracias. El Presidente diputado José de Jesús Zambrano Grijalva: Le sugiero, respetuosamente, diputado Santiago, que pueda canalizar desde luego su pre

In [36]:
list(re.split(r'|'.join(strings), all_text))[160]

' Valencia dice que la intervención metaconstitucional del presidente de la República para destituir al secretario de Seguridad Pública fue un día de tristeza y que hoy es un día de fiesta; pero hoy se está aprobando una intervención también metaconstitucional, no sólo del presidente de la República, sino del jefe de gobierno Miguel Ángel Mancera, para designar a diputados al congreso constituyente... '

In [24]:
 obtain_speaker_and_text2(list(re.split(r'|'.join(strings), all_text))[1])

NameError: name 'obtain_speaker_and_text2' is not defined

In [33]:
pd.DataFrame(all_interventions).speech[]

'«Sesión Constitutiva de la Cámara de Diputados de la LXIII Legislatura. Orden del día Sábado 29 de agosto de 2015. Informe de la Secretaría General e integración de la Mesa de Decanos. Declaración de quórum. Protesta constitucional del presidente de la Mesa de Decanos. Protesta constitucional de los diputados electos a la LXIII Legislatura. Elección de los integrantes de la Mesa Directiva que funcionará durante el primer año de ejercicio de la LXIII Legislatura. Declaración de la legal constitución de la Cámara de Diputados de la LXIII Legislatura. Cita a sesión de Congreso General. Designación de comisiones reglamentarias.» Se invita a los presentes a ponerse de pie. '

In [72]:
i = 'Blanca Margarita Cuata Domínguez (desde la curul). Lo mismo de la otra diputada, del artículo 3o. '

In [118]:
re.sub('[0-9]+[:][0-9]+', 'HORA', i)

' Valencia dice que la intervención metaconstitucional del presidente de la República para destituir al secretario de Seguridad Pública fue un día de tristeza y que hoy es un día de fiesta; pero hoy se está aprobando una intervención también metaconstitucional, no sólo del presidente de la República, sino del jefe de gobierno Miguel Ángel Mancera, para designar a diputados al congreso constituyente... '

In [119]:
all_text

'El Presidente diputado José de Jesús Zambrano Grijalva: Pido a la Secretaría haga del conocimiento de esta Presidencia el resultado del cómputo de asistencia de diputadas y diputados. La Secretaria diputada Alejandra Noemí Reynoso Sánchez:Se informa a la Presidencia que existen registrados previamente 428 diputadas y diputados, por lo tanto hay quórum, señor presidente. El Presidente diputado José de Jesús Zambrano Grijalva: (a las 12:53 horas): Se abre la sesión. El Presidente diputado José de Jesús Zambrano Grijalva: Consulte la Secretaría a la asamblea si se dispensa la lectura del orden del día. La Secretaria diputada Alejandra Noemí Reynoso Sánchez:Por instrucciones de la Presidencia, en votación económica se consulta a la asamblea si se dispensa la lectura al orden del día. Las diputadas y los diputados que estén por la afirmativa sírvanse manifestarlo. Las diputadas y los diputados que estén por la negativa sírvanse manifestarlo. Mayoría por la afirmativa, señor presidente. Se 

In [88]:
re.sub('::', ':', re.sub('[(].*[)]', ':',re.sub('[0-9]+[:][0-9]+', 'HORA', i))).split(':')

[' José de Jesús Zambrano Grijalva',
 ' Se levanta la sesión y se cita para la que tendrá lugar el próximo martes 29 de septiembre de 2015 a las 11 horas. Y se les informa que el registro de asistencia se hará en los lectores biométricos que estarán disponibles desde las HORA horas en el vestíbulo del salón de sesiones. ————— O ————— 79601c65-b9be-3386-a1a5-e39a2a0440f1']

In [25]:
def obtain_speaker_and_text2(x):
    speaker = re.sub('::', ':', re.sub('[(].*[)]', ':',re.sub('[0-9]+[:][0-9]+', 'HORA', i))).split(':')[0]
    text = re.sub('::', ':', re.sub('[(].*[)]', ':',re.sub('[0-9]+[:][0-9]+', 'HORA', i))).split(':')[1]
    speaker = re.sub('[(].*', '', speaker)
    speaker = re.sub('^[ ]+', '', speaker)
    return(speaker, text)

In [ ]:
def obtain_speaker_and_text(x):
    speaker = re.sub('[0-9]+[:][0-9]+', 'HORA', x).split(':')[0]
    text = re.sub('[0-9]+[:][0-9]+', 'HORA', x).split(':')[1]
    speaker = re.sub('[(].*', '', speaker)
    speaker = re.sub('^[ ]+', '', speaker)
    return(speaker, text)

In [39]:
all_text

'El Presidente diputado José de Jesús Zambrano Grijalva: \xa0Pido a la Secretaría haga del conocimiento de esta Presidencia el resultado del cómputo de asistencia de diputadas y diputados. La Secretaria diputada Ana Guadalupe Perea Santos: Se informa a la Presidencia que existen registrados previamente 462 diputadas y diputados. Por lo tanto hay quórum, señor presidente. El Presidente diputado José de Jesús Zambrano Grijalva(a las 11:28 horas): Gracias. Se abre la sesión El Presidente diputado José de Jesús Zambrano Grijalva: Consulte la Secretaría a la asamblea si se dispensa la lectura al orden del día. La Secretaria diputada Ana Guadalupe Perea Santos: Por instrucciones de la Presidencia, en votación económica se consulta a la asamblea si se dispensa la lectura del orden del día. Las diputadas y los diputados que estén por la afirmativa sírvanse manifestarlo. Gracias. Las diputadas y los diputados que estén por la negativa sírvanse manifestarlo. Gracias. Señor presidente, mayoría po

,speaker,speech
0,José de Jesús Zambrano Grijalva,Se pide a la secretaria de la honorable Cámar...
1,José de Jesús Zambrano Grijalva,Pido a la Secretaría haga del conocimiento de...
2,Ramón Bañales Arámbula,"Con gusto, señor Presidente. Se informa a est..."
3,José de Jesús Zambrano Grijalva,Se abre la sesión de Congreso General.
4,Ramón Bañales Arámbula,«Sesión de Congreso General.— Apertura del pri...
5,José de Jesús Zambrano Grijalva,El Congreso General de los Estados Unidos Me...
6,José de Jesús Zambrano Grijalva,Se va a proceder a entonar el Himno Nacional....
7,Ramón Bañales Arámbula,Gracias. Pueden tomar asiento.
8,José de Jesús Zambrano Grijalva,"Invito al senador Roberto Gil Zuarth, preside..."
9,José de Jesús Zambrano Grijalva,Se declara un receso. (Receso)


In [149]:
all_text = []
for i in obtain_text_sesion(df_data.link_debate[1]).values():
    all_text.append(i)
all_text = ' '.join(all_text)

10


In [ ]:
for i in df_data.link_debate:
    try:
        obtain_text_sesion(i)
    except:
        print(i)

In [150]:
all_text

'El Presidente diputado José de Jesús Zambrano Grijalva: Se pide a la secretaria de la honorable Cámara de Senadores, haga del conocimiento de esta Presidencia el resultado del registro de asistencia de senadoras y senadores. La Secretaria senadora Hilda Esthela Flores Escalera:Con su permiso, señor presidente. Le informo que con base en el registro previo de asistencia de la Secretaría, están presentes 90 ciudadanas senadoras y ciudadanos senadores. El Presidente diputado José de Jesús Zambrano Grijalva: Pido a la Secretaría haga del conocimiento de esta presidencia el resultado del registro de asistencia de diputadas y diputados. El Secretario diputado Ramón Bañales Arámbula: Con gusto, señor Presidente. Se informa a esta Presidencia que existen registrados previamente, 372 diputadas y diputados. Asimismo, señor presidente, le informo que hay una asistencia de 90 senadores y 372 diputados registrados hasta este momento. Hay quórum en el Congreso General. El Presidente diputado José d

In [6]:
df_speaker_speech(dict_session)

NameError: name 'dict_session' is not defined

In [64]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import time
import pandas as pd
import datetime
import uuid
from tqdm import tqdm
from datetime import date
from selenium.webdriver.chrome.options import Options

main_url = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid2.xql?legis=LXIIV#'

options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get(main_url)

partido = driver.find_elements_by_class_name('Grupos-Menu-subItem')[0].text
driver.find_elements_by_class_name('Grupos-Menu-subItem')[0].click()

names_deputies_party = []
partido_list = []
for i in driver.find_element_by_id('selectDipu').find_elements_by_css_selector('option'):
    names_deputies_party.append(i.text)
    partido_list.append(partido)

driver.find_element_by_id('selectDipu').find_elements_by_css_selector('option')[2].click()

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


IndexError: list index out of range